# 2절. 트랜잭션 데이터

In [3]:
import csv
with open("basket.csv", "r", encoding="utf-8") as cf:
    transactions = []
    r = csv.reader(cf)
    for row in r:
        transactions.append(row)

In [4]:
transactions

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

# 3절. 연관분석

In [8]:
from apyori import apriori
rules = apriori(transactions, min_support=0.1, min_confidence=0.1)
results = list(rules)

In [14]:
results[0]

RelationRecord(items=frozenset({'맥주'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주'}), confidence=0.4, lift=1.0)])

In [12]:
results[10]

RelationRecord(items=frozenset({'소주', '콜라'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주', '콜라'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

In [15]:
import pandas as pd
result_df = pd.DataFrame(None, columns=["LHS", "RHS", "support", "confidence", "lift"])

index = 0

for result in results:
    support = result[1]
    for ordered_item in result[2]:
        lhs = ",".join(ordered_item[0])
        rhs = ",".join(ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        result_df.loc[index] = [lhs, rhs, support, confidence, lift]
        index += 1

In [19]:
result_df.head()

,LHS,RHS,support,confidence,lift
0,,맥주,0.4,0.4,1.0
1,,소주,0.6,0.6,1.0
2,,오렌지주스,0.4,0.4,1.0
3,,와인,0.6,0.6,1.0
4,,콜라,0.8,0.8,1.0


In [18]:
result_df[(result_df["LHS"]=="오렌지주스") & (result_df["RHS"]=="와인")]

,LHS,RHS,support,confidence,lift
24,오렌지주스,와인,0.2,0.5,0.833333


# 4절. 뉴스 기사 연관분석 실습

## 뉴스 링크 가져오기

In [24]:
import requests
url = "https://fs.jtbc.joins.com//RSS/sports.xml"
news = requests.get(url)

In [26]:
from bs4 import BeautifulSoup
sport_news = BeautifulSoup(news.content, "xml")
sport_news_list = sport_news.select("item > link")

In [28]:
sport_news_list

[<link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041616</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041638</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041646</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041514</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041512</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041513</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041376</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041377</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041345</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041372</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041238</link>,
 <link>https://news.jtbc.joins.com/article/article.aspx?news_id=NB12041237</link>,
 <li

## 뉴스 기사 형태소 분석

In [42]:
from konlpy.tag import Kkma
kkma = Kkma()
news_filter = [] #transaction
for link in sport_news_list:
    news_url = link.text
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.content, "html.parser")
    news_content = news_soup.select_one("#articlebody > .article_content")
    news_filter.append(list(filter(lambda word: len(word)>1, kkma.nouns(news_content.text))))

In [43]:
from apyori import apriori
rules = apriori(news_filter, min_support=0.3, min_confidence=0.3)
results = list(rules)

KeyboardInterrupt: 

In [ ]:
results.head()

In [ ]:
import pandas as pd
result_df = pd.DataFrame(None, columns=["LHS", "RHS", "support", "confidence", "lift"])
index = 0
for row in results:
    support = row[1]
    ordered_stat = row[2]
    for ordered_item in ordered_stat:
        lhs = " ".join(ordered_item[0])
        rhs = " ".join(ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        result_df.loc[index] = [lhs, rhs, support, confidence, lift]
        index += 1

In [ ]:
result_df.loc[result_df.lhs==""].sort_values(by="support", ascending=False)

# 5절. 연습문제

In [44]:
import csv
with open("mybasket.csv", "r", encoding="utf-8") as cf:
    transactions = []
    r = csv.reader(cf)
    for row in r:
        transactions.append(row)

In [46]:
len(transactions)

786

In [47]:
from apyori import apriori
rules = apriori(transactions, min_support=0.1, min_confidence=0.1)
results = list(rules)

In [49]:
len(results)

53

In [50]:
import pandas as pd
result_df = pd.DataFrame(None, columns=["LHS", "RHS", "support", "confidence", "lift"])
index=0

for row in results:
    support = row[1]
    ordered_stat = row[2]
    for ordered_item in ordered_stat:
        lhs = " ".join(x.strip() for x in ordered_item[0])
        rhs = " ".join(x.strip() for x in ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        result_df.loc[index] = [lhs, rhs, support, confidence, lift]
        index += 1

In [52]:
#향상도가 가장 높은 연관 규칙
result_df.sort_values("lift", ascending=False)

,LHS,RHS,support,confidence,lift
179,bakery deco,milk,0.100509,0.441341,2.343877
178,milk,bakery deco,0.100509,0.533784,2.343877
244,bakery alcohol,clothes frozen,0.104326,0.485207,2.283669
249,clothes frozen,bakery alcohol,0.104326,0.491018,2.283669
231,alcohol deco,bakery clothes,0.100509,0.580882,2.271510
...,...,...,...,...,...
210,,clothes frozen snack,0.128499,0.128499,1.000000
41,clothes,deco,0.216285,0.440415,0.966944
42,deco,clothes,0.216285,0.474860,0.966944
15,deco,alcohol,0.173028,0.379888,0.960103


In [59]:
# 가장 많이 팔린 두 제품
result_df[result_df["LHS"]==""].sort_values(by="support", ascending=False)

,LHS,RHS,support,confidence,lift
2,,clothes,0.491094,0.491094,1.0
6,,snack,0.474555,0.474555,1.0
3,,deco,0.455471,0.455471,1.0
1,,bakery,0.428753,0.428753,1.0
4,,frozen,0.404580,0.404580,1.0
0,,alcohol,0.395674,0.395674,1.0
25,,bakery clothes,0.255725,0.255725,1.0
49,,clothes snack,0.244275,0.244275,1.0
37,,bakery snack,0.232824,0.232824,1.0
16,,alcohol frozen,0.230280,0.230280,1.0


In [60]:
# 가장 많이 팔린 두 제품 사이의 신뢰도와 향상도
result_df[(result_df["LHS"]=="clothes") & (result_df["RHS"]=="snack")]
result_df[(result_df["LHS"]=="snack") & (result_df["RHS"]=="clothes")]

,LHS,RHS,support,confidence,lift
50,clothes,snack,0.244275,0.497409,1.04816


In [63]:
result_df[(result_df["LHS"]=="snack") & (result_df["RHS"]=="clothes")]

,LHS,RHS,support,confidence,lift
51,snack,clothes,0.244275,0.514745,1.04816


In [61]:
# 두 제품을 포함하는 제품들 사이의 신뢰도와 향상도
result_df[(result_df["LHS"].str.contains("clothes")) & (result_df["RHS"]=="snack")]

,LHS,RHS,support,confidence,lift
50,clothes,snack,0.244275,0.497409,1.048160
116,alcohol clothes,snack,0.136132,0.636905,1.342110
165,bakery clothes,snack,0.147583,0.577114,1.216118
207,deco clothes,snack,0.138677,0.641176,1.351112
214,clothes frozen,snack,0.128499,0.604790,1.274438
265,bakery alcohol clothes,snack,0.101781,0.695652,1.465905


In [62]:
result_df[(result_df["LHS"].str.contains("snack")) & (result_df["RHS"]=="clothes")]

,LHS,RHS,support,confidence,lift
51,snack,clothes,0.244275,0.514745,1.048160
117,alcohol snack,clothes,0.136132,0.618497,1.259427
166,bakery snack,clothes,0.147583,0.633880,1.290750
209,deco snack,clothes,0.138677,0.619318,1.261099
216,frozen snack,clothes,0.128499,0.597633,1.216942
266,bakery alcohol snack,clothes,0.101781,0.740741,1.508348


In [ ]:
# 두 제품을 포함하는 제품들 사이의 신뢰도와 향상도
result_df[(result_df["LHS"].str.contains("clothes")) & (result_df["RHS"]=="snack")]
result_df[(result_df["LHS"].str.contains("snack")) & (result_df["RHS"]=="clothes")]